# Prerequisites

In [ ]:
import numpy as np
import pandas as pd
from tm import StockDataProvider
from tm.trading_rules import SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC
from tm.optimizers import GeneticOptimizer, StrategyPerformanceEvaluator, map_chromosome_to_trading_rule_parameters, filter_for_active_rules, calculate_absolute_buy_and_hold_returns
from tm.backtesting import GeometricBrownianMotion, MonteCarloCrossValidation
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Stock Price Simulation with Geometric Brownian Motion

In [ ]:
data = StockDataProvider('MSFT', start='2019-01-01', end='2019-12-31')

In [ ]:
gbm = GeometricBrownianMotion(data)
simulations_df = gbm.simulate(num_simulations=100, time_steps=252)

In [ ]:
real_stock_prices = StockDataProvider('MSFT', start='2020-01-01').history['Close']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10))
data.history['Close'].plot(ax=ax, label='Daily stock prices used for simulation', color='blue')
simulations_df.plot(ax=ax, style=':', legend=False)
real_stock_prices.plot(ax=ax, label='Real stock prices', lw=5, color='blue')
ax.set_title('MSFT stock price Monte Carlo Simulation using Geometric Brownian Motion stochastic process');

# Optimization and Backtesting

In [ ]:
trading_rules = [SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC]

Run the optimization with the Genetic Algorithm

In [ ]:
optimizer = GeneticOptimizer(data, trading_rules)

In [ ]:
%%time
hof = optimizer.run(pop_size=500, ngen=10, hof_size=50)

Use 100-fold crossvalidation on artificially generated stock prices with geometric brownian motion

In [ ]:
crossvalidator = MonteCarloCrossValidation(hof, gbm, trading_rules)

In [ ]:
%%time
# Best individual is the bitvector representing the best strategy after crossvalidation
best_individual = crossvalidator.run(num_iterations=100, time_steps=252)

Apply the best found strategy on the real data and compare the result with buy and hold

In [ ]:
real_data = StockDataProvider('MSFT', start='2020-01-01')
rule_instances = list(map(lambda Rule, params: Rule(real_data, *params), trading_rules, map_chromosome_to_trading_rule_parameters(best_individual, trading_rules)))
active_rule_instances = filter_for_active_rules(best_individual, rule_instances)
evaluator = StrategyPerformanceEvaluator(active_rule_instances)
print('Active rules:', list(map(lambda rule: rule.__class__.__name__, active_rule_instances)))
print('Active rules parameters:', list(map(lambda param: param[1], filter(lambda param: trading_rules[param[0]] in map(lambda rule: rule.__class__, active_rule_instances), enumerate(map_chromosome_to_trading_rule_parameters(best_individual, trading_rules))))))
print('Net profit of strategy:', evaluator.calculate_net_profit())
print('Net profit of buy and hold:', calculate_absolute_buy_and_hold_returns(real_data))